In [3]:
# Import libraries
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the model (adjust path to where your files are in Drive)
model = tf.keras.models.load_model('/content/housing_lstm_model.keras')
print("Model loaded successfully!")

# Load scaler parameters
scaler_params = np.load('/content/scaler_params.npy', allow_pickle=True).item()

# Reconstruct the scaler
scaler = MinMaxScaler()
scaler.scale_ = scaler_params['scale_']
scaler.min_ = scaler_params['min_']
scaler.data_min_ = scaler_params['data_min_']
scaler.data_max_ = scaler_params['data_max_']
print("Scaler loaded successfully!")

# Test prediction
sample_input = np.array([100000] * 10).reshape(-1, 1)  # 10 timestamps of same price
scaled_input = scaler.transform(sample_input)
scaled_input = scaled_input.reshape(1, 10, 1)  # Reshape for LSTM

prediction = model.predict(scaled_input, verbose=0)
prediction = scaler.inverse_transform(prediction)
print(f"Test prediction: ${prediction[0][0]:,.2f}")


Model loaded successfully!
Scaler loaded successfully!
Test prediction: $73,281.41


In [4]:
!pip install onnx onnxruntime numpy scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


In [7]:
import onnxruntime as ort
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load ONNX model (adjust path as needed)
model_path = "/content/housing_lstm_model.onnx"
sess = ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])

# Load scaler parameters
scaler_params = np.load('/content/scaler_params.npy', allow_pickle=True).item()

# Reconstruct scaler
scaler = MinMaxScaler()
scaler.scale_ = scaler_params['scale_']
scaler.min_ = scaler_params['min_']
scaler.data_min_ = scaler_params['data_min_']
scaler.data_max_ = scaler_params['data_max_']

# Create test input
test_prices = np.array([100000] * 10).reshape(-1, 1)
scaled_input = scaler.transform(test_prices)
scaled_input = scaled_input.reshape(1, 10, 1).astype(np.float32)

# Make prediction
input_name = sess.get_inputs()[0].name
prediction = sess.run(None, {input_name: scaled_input})[0]

# Convert prediction back to price
final_prediction = scaler.inverse_transform(prediction)
print(f"Predicted price: ${final_prediction[0][0]:,.2f}")

Predicted price: $73,281.41


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
import gradio as gr
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load model and scaler
def load_model_and_scaler():
    model = tf.keras.models.load_model('housing_lstm_model.keras')
    scaler_params = np.load('scaler_params.npy', allow_pickle=True).item()

    scaler = MinMaxScaler()
    scaler.scale_ = scaler_params['scale_']
    scaler.min_ = scaler_params['min_']
    scaler.data_min_ = scaler_params['data_min_']
    scaler.data_max_ = scaler_params['data_max_']

    return model, scaler

def predict_price(price1, price2, price3, price4, price5, price6, price7, price8, price9, price10):
    # Get inputs into array
    prices = np.array([price1, price2, price3, price4, price5,
                      price6, price7, price8, price9, price10])

    # Scale input
    scaled_input = scaler.transform(prices.reshape(-1, 1))
    scaled_input = scaled_input.reshape(1, 10, 1)

    # Make prediction
    prediction = model.predict(scaled_input, verbose=0)

    # Inverse transform prediction
    final_prediction = scaler.inverse_transform(prediction)[0][0]

    return f"${final_prediction:,.2f}"

# Load model and scaler globally
model, scaler = load_model_and_scaler()

# Create Gradio interface
iface = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Number(label=f"Price {i+1}") for i in range(10)
    ],
    outputs=gr.Textbox(label="Predicted Next Price"),
    title="Housing Price Prediction",
    description="Enter 10 consecutive house prices to predict the next price",
    examples=[
        [100000, 102000, 105000, 103000, 106000, 108000, 110000, 112000, 115000, 118000],
        [200000, 205000, 208000, 210000, 215000, 218000, 220000, 225000, 228000, 230000]
    ]
)

if __name__ == "__main__":
    iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ee3752be08e07fd88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
